# 基礎建設

In [1]:
import pandas as pd
import gzip, json
import sklearn
import numpy as np

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

C:\Users\User\.conda\envs\spacy_env\lib\site-packages\ipykernel\parentpoller.py:109: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


In [2]:
metadata = getDF('meta_All_Beauty.json.gz')
ratings = pd.read_csv('All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [3]:
metadata

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32887,[],,[],,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,,[],"2,145,325 in Beauty & Personal Care (",[],{'ASIN: ': 'B01HIWLLUK'},All Beauty,,,$9.95,B01HIWLLUK,[],[]
32888,[],,[],,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],,Salon Perfect,[],"1,639,713 in Beauty & Personal Care (",[],"{'ASIN: ': 'B01HJ1K3YK', 'UPC:': '671635851871'}",All Beauty,,,,B01HJ1K3YK,[],[]
32889,[],,[],,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,"[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",,,[],"207,410 in Beauty & Personal Care (","[B01KON9B4S, B000JN4CR0, B071ZHMRHS, B01HJ84TN...","{'Shipping Weight:': '1 pounds (', 'ASIN: ': '...",All Beauty,,,$55.63,B01HJ84SGM,[],[]
32890,[],,[Brand new and high quality<br> Enables fast v...,,12 White Feather Shuttlecocks Birdies Badminto...,[],,GBSTORE,[],"965,673 in Beauty & Personal Care (",[],"{'Shipping Weight:': '4.8 ounces (', 'ASIN: ':...",All Beauty,,,$12.99,B01HJASD20,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


# 資料整理

In [4]:
metadata['ranking'] = metadata['rank'].str.split(expand = True)[0]
metadata['ranking'] = metadata['ranking'].str.replace(',', '')
metadata['ranking'] = pd.to_numeric(metadata['ranking'])

In [5]:
metadata['sub_category'] = metadata['rank'].str.split('in ').str[1]
metadata['sub_category'] = metadata['sub_category'].str.replace(r'&amp;', '', regex=True)
metadata['sub_category'] = metadata['sub_category'].str.replace(r'\(', '', regex=True)
metadata['sub_category'] = metadata['sub_category'].str.replace(r'&', '', regex=True)
print(metadata['sub_category'].value_counts())
metadata = metadata.drop(metadata[(metadata['sub_category'] != 'Beauty  Personal Care ')].index)
metadata

Beauty  Personal Care        32380
Grocery  Gourmet Food           38
Health  Household               21
Sports  Outdoors                 3
Toys  Games                      3
Baby                             2
Clothing, Shoes  Jewelry         2
Home  Kitchen                    1
Automotive                       1
Tools  Home Improvement          1
Name: sub_category, dtype: int64


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,...,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,ranking,sub_category
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",...,{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[],2938573.0,Beauty Personal Care
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",...,"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[],872854.0,Beauty Personal Care
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (",...,"{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[],956696.0,Beauty Personal Care
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",...,"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1870258.0,Beauty Personal Care
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (",...,"{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,67701.0,Beauty Personal Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32887,[],,[],,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,,[],"2,145,325 in Beauty & Personal Care (",...,{'ASIN: ': 'B01HIWLLUK'},All Beauty,,,$9.95,B01HIWLLUK,[],[],2145325.0,Beauty Personal Care
32888,[],,[],,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],,Salon Perfect,[],"1,639,713 in Beauty & Personal Care (",...,"{'ASIN: ': 'B01HJ1K3YK', 'UPC:': '671635851871'}",All Beauty,,,,B01HJ1K3YK,[],[],1639713.0,Beauty Personal Care
32889,[],,[],,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,"[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",,,[],"207,410 in Beauty & Personal Care (",...,"{'Shipping Weight:': '1 pounds (', 'ASIN: ': '...",All Beauty,,,$55.63,B01HJ84SGM,[],[],207410.0,Beauty Personal Care
32890,[],,[Brand new and high quality<br> Enables fast v...,,12 White Feather Shuttlecocks Birdies Badminto...,[],,GBSTORE,[],"965,673 in Beauty & Personal Care (",...,"{'Shipping Weight:': '4.8 ounces (', 'ASIN: ':...",All Beauty,,,$12.99,B01HJASD20,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,965673.0,Beauty Personal Care


In [6]:
metadata['text_data'] = metadata.agg(lambda x: f'{x["title"]}, {" ".join(x["description"])}', axis=1)
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
text = metadata['text_data'].tolist()

In [7]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.cluster import KElbowVisualizer

In [27]:
vectorizor = TfidfVectorizer(stop_words='english', min_df=150)
for idx, value in enumerate(text):
    if not isinstance(value, str):
        text[idx] = ''
feature = vectorizor.fit_transform(text).toarray()
print(len(vectorizor.get_feature_names()))

957


C:\Users\User\.conda\envs\spacy_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
model = AgglomerativeClustering(n_clusters=20)
result = model.fit_predict(feature)

In [29]:
print(result)

[ 2 12 12 ... 12 15 11]


In [30]:
metadata['specify'] = pd.Series(result)
metadata['specify'].value_counts()
def transform(data):
    res = dict()
    for item, row in data.iterrows():
        res[row['asin']] = row['specify']
    return res
cluster_dict = transform(metadata)
def construct(data):
    if cluster_dict.get(data):
        return cluster_dict[data]
    else:
        return 21.0
ratings['cluster'] = ratings['asin'].apply(construct)
ratings['cluster'].value_counts()

12.000    129309
2.000      91983
1.000      18403
21.000     16489
17.000     15954
11.000     13425
16.000     12858
15.000     10510
5.000       8888
6.000       8229
4.000       7871
3.000       7416
7.000       6277
9.000       5127
13.000      4574
10.000      3172
8.000       2878
14.000      2323
18.000      1830
19.000       533
Name: cluster, dtype: int64

In [31]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [32]:
print(ratings_trainings)
ratings_trainings[(ratings_trainings['overall']>ratings_trainings['overall'].mean())]

              asin      reviewerID  overall  unixReviewTime       DATE  \
0       0143026860  A1V6B6TNIC10QE    1.000      1424304000 2015-02-19   
1       0143026860  A2F5GHSXFQ0W6J    4.000      1418860800 2014-12-18   
2       0143026860  A1572GUYS7DGSR    4.000      1407628800 2014-08-10   
3       0143026860   A1PSGLFK1NSVO    5.000      1362960000 2013-03-11   
4       0143026860   A6IKXKZMTKGSC    5.000      1324771200 2011-12-25   
...            ...             ...      ...             ...        ...   
371340  B01HJEGTYK  A202DCI7TV1022    1.000      1500508800 2017-07-20   
371341  B01HJEGTYK  A3FSOR5IJOFIBE    5.000      1489622400 2017-03-16   
371342  B01HJEGTYK  A1B5DK6CTP2P24    5.000      1488326400 2017-03-01   
371343  B01HJEGTYK  A23OUYS5IRMJS9    2.000      1487635200 2017-02-21   
371344  B01HJEGTYK  A24KQ9RVU81L87    2.000      1481760000 2016-12-15   

        cluster  
0        21.000  
1        21.000  
2        21.000  
3        21.000  
4        21.000  
...

,asin,reviewerID,overall,unixReviewTime,DATE,cluster
3,0143026860,A1PSGLFK1NSVO,5.000,1362960000,2013-03-11,21.000
4,0143026860,A6IKXKZMTKGSC,5.000,1324771200,2011-12-25,21.000
5,0143026860,A36NF437WZLQ9E,5.000,1267142400,2010-02-26,21.000
8,014789302X,AQ812VYVTC2RJ,5.000,1483401600,2017-01-03,21.000
11,014789302X,A1NLF2JD7BVOB4,5.000,1474848000,2016-09-26,21.000
...,...,...,...,...,...,...
371333,B01HJ84SGM,A1IYJVHJSCR8DR,5.000,1496016000,2017-05-29,NaN
371334,B01HJ84SGM,A7WHO9ZFPHU2B,5.000,1494288000,2017-05-09,NaN
371338,B01HJEGTYK,A338U2R53FLF9U,5.000,1509148800,2017-10-28,NaN
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.000,1489622400,2017-03-16,NaN


In [33]:
def recommender(training_data, time, users=[], k=10):

    target = training_data[
                (training_data['DATE']>=time)&
                (training_data['overall']>=training_data['overall'].mean())]
    brand_2 = target['asin'].value_counts().sort_values(ascending=False).keys()[:k]
    recommendations = {user: brand_2.tolist() for user in users}
    return recommendations

def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
  * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
  * ratings_by_user: dict 利用訓練資料學習的推薦商品
  * method: str
  * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

time_list = ['2018-07-01','2018-05-01', '2018-01-01', '2016-01-01']
origin_score = list()
for time in time_list:
    ratings_by_user = recommender(ratings_trainings, time, users)
    score = evaluate(ratings_testings_by_user, ratings_by_user)
    print(f'{time}, score: {score}')
    origin_score.append(score)

2018-07-01, score: 0.13728813559322034
2018-05-01, score: 0.1152542372881356
2018-01-01, score: 0.1
2016-01-01, score: 0.08305084745762711


In [34]:
def recommender(training_data, group, time, users=[], k=10):
    if group > 19 :
        group = 20
    target = training_data[(training_data['cluster'] == group)&
                (training_data['DATE']>=time)&
                (training_data['overall']>=training_data['overall'].mean())]
    brand_2 = target['asin'].value_counts().sort_values(ascending=False).keys()[:k]
    recommendations = {user: brand_2.tolist() for user in users}
    return recommendations

score_list = []
product_list = []
for time in time_list:
    print(f'time: {time}')
    tmp = list()
    product = list()
    for i in range(21):
        ratings_by_user = recommender(ratings_trainings, i, time, users)
        score = evaluate(ratings_testings_by_user, ratings_by_user)
        tmp.append(score)
        product.append(ratings_by_user['A100XQFWKQ30O2'])
        print(f'{i}, score: {score}')
    product_list.append(product)
    score_list.append(tmp)
    print('='*50)

time: 2018-07-01
0, score: 0.0
1, score: 0.006779661016949152
2, score: 0.06271186440677966
3, score: 0.005084745762711864
4, score: 0.03220338983050847
5, score: 0.03559322033898305
6, score: 0.0
7, score: 0.015254237288135594
8, score: 0.0
9, score: 0.006779661016949152
10, score: 0.0288135593220339
11, score: 0.01864406779661017
12, score: 0.10508474576271186
13, score: 0.0
14, score: 0.0
15, score: 0.04745762711864407
16, score: 0.005084745762711864
17, score: 0.03220338983050847
18, score: 0.001694915254237288
19, score: 0.0
20, score: 0.0
time: 2018-05-01
0, score: 0.0
1, score: 0.006779661016949152
2, score: 0.05084745762711865
3, score: 0.003389830508474576
4, score: 0.03220338983050847
5, score: 0.030508474576271188
6, score: 0.006779661016949152
7, score: 0.015254237288135594
8, score: 0.0
9, score: 0.003389830508474576
10, score: 0.0288135593220339
11, score: 0.01864406779661017
12, score: 0.09322033898305085
13, score: 0.0
14, score: 0.0
15, score: 0.04745762711864407
16, s

In [36]:
pd.options.display.float_format = '{:,.3f}'.format
table = pd.DataFrame(score_list, index=time_list)
table

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
2018-07-01,0.000,0.007,0.063,0.005,0.032,0.036,0.000,0.015,0.000,0.007,...,0.019,0.105,0.000,0.000,0.047,0.005,0.032,0.002,0.000,0.000
2018-05-01,0.000,0.007,0.051,0.003,0.032,0.031,0.007,0.015,0.000,0.003,...,0.019,0.093,0.000,0.000,0.047,0.007,0.027,0.002,0.000,0.000
2018-01-01,0.000,0.007,0.025,0.003,0.031,0.036,0.008,0.015,0.000,0.005,...,0.022,0.092,0.000,0.000,0.047,0.007,0.015,0.002,0.000,0.000
2016-01-01,0.000,0.002,0.000,0.002,0.032,0.029,0.002,0.008,0.000,0.003,...,0.007,0.083,0.000,0.000,0.032,0.005,0.015,0.003,0.000,0.000


In [37]:
ori_table = pd.DataFrame(origin_score, index=time_list)
ori_table

,0
2018-07-01,0.137
2018-05-01,0.115
2018-01-01,0.100
2016-01-01,0.083
